# Скачивание и импорт библиотек

Предварительно необходимо установить на локальный хост chromedriver - https://chromedriver.chromium.org/

In [1]:
!pip install selenium
!pip install beautifulsoup4
!pip install lxml

In [58]:
import selenium
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import pandas as pd

In [3]:
print("Установленная версия библиотеки Selenium:", selenium.__version__)

Установленная версия библиотеки Selenium: 4.15.2


In [4]:
# path_to_chromedriver = "D:\\jupyter\\Parsing_selenium\\chromedriver.exe"

# Работа с браузером

In [5]:
browser = Chrome()

## Перейдём на стартовую страницу

In [6]:
url = "https://bus.gov.ru/registry"

In [7]:
browser.get(url)

In [8]:
browser.implicitly_wait(0.5)

Код страницы получаем через атрибут page_source.

In [9]:
browser.page_source

'<html lang="en" class=""><head><style type="text/css">[uib-typeahead-popup].dropdown-menu{display:block;}</style><style type="text/css">.uib-time input{width:50px;}</style><style type="text/css">[uib-tooltip-popup].tooltip.top-left > .tooltip-arrow,[uib-tooltip-popup].tooltip.top-right > .tooltip-arrow,[uib-tooltip-popup].tooltip.bottom-left > .tooltip-arrow,[uib-tooltip-popup].tooltip.bottom-right > .tooltip-arrow,[uib-tooltip-popup].tooltip.left-top > .tooltip-arrow,[uib-tooltip-popup].tooltip.left-bottom > .tooltip-arrow,[uib-tooltip-popup].tooltip.right-top > .tooltip-arrow,[uib-tooltip-popup].tooltip.right-bottom > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.top-left > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.top-right > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.bottom-left > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.bottom-right > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.left-top > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.left-bottom > .toolt

## Поиск поля

На сайте хотим искать информацию по полю 'Реквизиты организации', например, набрать в этом поле 'онкологический диспансер'. Найдем необходимое поле с помощью метода .find_...

<input _ngcontent-sgl-c404="" type="text" name="agency" placeholder="Введите наименование организации, код ПГМУ, код Сводного реестра, ИНН, ОГРН" class="search-input search-input_big ng-pristine ng-valid ng-touched" style="margin-bottom: 0;" wfd-id="id0">

### Поиск по тегу

In [10]:
browser.find_element(by=By.TAG_NAME, value="input")

<selenium.webdriver.remote.webelement.WebElement (session="58cff7b0c453902a9b47821026b8d6fd", element="B0ACCFF5FB342BE43FC32349B2644D21_element_29")>

### Поиск по классу

Можно искать только по одному имени класса.

In [11]:
browser.find_element(by=By.CLASS_NAME, value="search-input")

<selenium.webdriver.remote.webelement.WebElement (session="58cff7b0c453902a9b47821026b8d6fd", element="B0ACCFF5FB342BE43FC32349B2644D21_element_29")>

### Поиск по Xpath

Xpath копируем со странички с нужного элемента.

In [12]:
browser.find_element(by=By.XPATH, value="/html/body/div[2]/ui-view/ui-view-ng-upgrade/ui-view/app-registry/app-registry-filter/form/div[1]/div[1]/label/div[2]/input")

<selenium.webdriver.remote.webelement.WebElement (session="58cff7b0c453902a9b47821026b8d6fd", element="B0ACCFF5FB342BE43FC32349B2644D21_element_29")>

### Поиск по CSS селектору

In [13]:
browser.find_element(by=By.CSS_SELECTOR, value="body > div.main > ui-view > ui-view-ng-upgrade > ui-view > app-registry > app-registry-filter > form > div:nth-child(1) > div:nth-child(1) > label > div.search__col.search__col_type_2 > input")

<selenium.webdriver.remote.webelement.WebElement (session="58cff7b0c453902a9b47821026b8d6fd", element="B0ACCFF5FB342BE43FC32349B2644D21_element_29")>

### Поиск по тексту - Поиск ссылки, лежащей под определнным текстом

Будем искать ссылку, по которой идёт переход при нажатии кнопки со словами "Открытые данные".

In [14]:
browser.find_element(by=By.LINK_TEXT, value="Открытые данные").get_attribute('href')

'https://bus.gov.ru/open-data'

# Передача информации в поле

## Заполнение поля

Сохраним в переменную input_tab нужное нам для заполнения поле.

In [15]:
input_tab = browser.find_element(by=By.TAG_NAME, value="input")
input_tab

<selenium.webdriver.remote.webelement.WebElement (session="58cff7b0c453902a9b47821026b8d6fd", element="B0ACCFF5FB342BE43FC32349B2644D21_element_29")>

Заполняем с помощью метода .send_keys().

In [16]:
input_tab.send_keys('онкологический диспансер')

## Имитация нажатия кнопки

Имитируем нажатие кнопки "Показать". Находим её по XPATH, который написали самостоятельно. Затем кликаем по ней.

<button _ngcontent-sgl-c404="" type="submit" class="button button_size_small float-l">Показать</button>

In [17]:
button = browser.find_element(by=By.XPATH, value="//button[@type='submit']")
button

<selenium.webdriver.remote.webelement.WebElement (session="58cff7b0c453902a9b47821026b8d6fd", element="B0ACCFF5FB342BE43FC32349B2644D21_element_32")>

In [18]:
# button.click()

Можно было просто имитировать нажатие Enter.

In [19]:
input_tab.send_keys(Keys.ENTER)

# Сбор информации

Будем собирать следующую информацию: название компании и ссылку на регистрационные данные. Будем использовать BeautifulSoup.

In [26]:
soup = BeautifulSoup(browser.page_source, 'lxml')

## Одна организация

<a _ngcontent-voo-c396="" target="_blank" class="result__title ng-star-inserted" href="/info-card/271599"> АВТОНОМНОЕ УЧРЕЖДЕНИЕ ЧУВАШСКОЙ РЕСПУБЛИКИ "РЕСПУБЛИКАНСКИЙ КЛИНИЧЕСКИЙ ОНКОЛОГИЧЕСКИЙ ДИСПАНСЕР" МИНИСТЕРСТВА ЗДРАВООХРАНЕНИЯ ЧУВАШСКОЙ РЕСПУБЛИКИ </a>

In [30]:
name = soup.find('a', class_="result__title ng-star-inserted").text.strip()
name

'АВТОНОМНОЕ УЧРЕЖДЕНИЕ ЧУВАШСКОЙ РЕСПУБЛИКИ "РЕСПУБЛИКАНСКИЙ КЛИНИЧЕСКИЙ ОНКОЛОГИЧЕСКИЙ ДИСПАНСЕР" МИНИСТЕРСТВА ЗДРАВООХРАНЕНИЯ ЧУВАШСКОЙ РЕСПУБЛИКИ'

<a _ngcontent-sgl-c396="" target="_blank" class="result__button result__button_registry result__button_href-button" href="/agency/531369/register-info"> Регистрационные данные </a>

In [33]:
link = url + soup.find('a', class_="result__button result__button_registry result__button_href-button").get("href")
link

'https://bus.gov.ru/registry/agency/22908/register-info'

In [ ]:
browser.quit()

## Одна страница

In [35]:
len(soup.findAll('div', class_="result"))

10

In [36]:
len(browser.find_elements(by=By.CLASS_NAME, value="result"))

10

Соберем всё, что было.

In [72]:
browser = Chrome()
url = "https://bus.gov.ru/registry"
browser.get(url)
browser.implicitly_wait(0.5)

input_tab = browser.find_element(by=By.TAG_NAME, value="input")

input_tab.send_keys('онкологический диспансер')
input_tab.send_keys(Keys.ENTER)

sleep(2)

soup = BeautifulSoup(browser.page_source, 'lxml')

orgs = soup.findAll('div', class_="result")

data = []

In [73]:
for org in orgs:
    name = org.find('a', class_="result__title ng-star-inserted").text.strip()
    link = url + org.find('a', class_="result__button result__button_registry result__button_href-button").get("href")
    data.append([name, link])

browser.quit()

In [74]:
len(data)

10

## Несколько страниц

In [116]:
data = []

In [117]:
browser = Chrome()
url = "https://bus.gov.ru/registry"
browser.get(url)
browser.implicitly_wait(0.5)

input_tab = browser.find_element(by=By.TAG_NAME, value="input")

input_tab.send_keys('онкологический диспансер')
input_tab.send_keys(Keys.ENTER)

sleep(3)

for page in range(10):
    soup = BeautifulSoup(browser.page_source, 'lxml')

    orgs = soup.findAll('div', class_="result")

    for org in orgs:
        name = org.find('a', class_="result__title ng-star-inserted").text.strip()
        link = url + org.find('a', class_="result__button result__button_registry result__button_href-button").get("href")
        data.append([name, link])
        
    # кликаем на кнопку следующая страница
    # <div _ngcontent-akr-c123="" class="info-section-paginator__next ng-star-inserted">следующая</div>
    
    try:
        browser.find_element(by=By.CLASS_NAME, value="info-section-paginator__next").click()
    except:
        print(page+1)
        print('end')
        break
    sleep(12)
    print(page+1)
    
browser.quit()

1
2
3
4
5
6
7
8
9
10
end


In [118]:
len(data)

92

In [119]:
columns_names = ['name', 'link']
df = pd.DataFrame(data, columns=columns_names)

In [124]:
!pip freeze > requirements.txt